# Анкета 2.0
### Библиотеки

In [1]:
from docxtpl import DocxTemplate, InlineImage
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.shared import Cm
import numpy as np
import pandas as pd 
from math import pi
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import requests
import re
from string import ascii_letters
import ast
import docx2txt
import json
import docx
import os
import time
import datetime
import PySimpleGUI as sg
from string import punctuation
from pymystem3 import Mystem
from stop_words import get_stop_words
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
import matplotlib.pyplot as plt
import requests
from pandas.io.json import dumps
from tqdm import tqdm
import datetime

### Диск

In [2]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=922428539074-3lcp6a62nd735nvdh38lj548n223snd0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


### ID папки с таблицами и видеовопросами

In [3]:
tables_root_id = drive.ListFile({'q': "title = 'Таблицы -> Анкеты'"}).GetList()[0]['id']
videoquest_folder_names = ['Вопрос от Ивана Федосеева', 'Вопрос от Ольги Орловой', 'Вопрос от Катерины Ходуновой', 'Вопрос от Олега Лысака']
videoquest_root_ids = [drive.ListFile({'q': "title = '{}'".format(name)}).GetList()[0]['id'] for name in videoquest_folder_names]

### Все таблицы с данными

In [4]:
tables = []
file_list = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % tables_root_id}).GetList()
for file in file_list:
    if file['title'] == 'Шаблон отчета v.4.0a.docx':
    #if file['title'] == 'Шаблон отчета v.4.0.docx':
        print(file['title'])
        file.GetContentFile(file['title'])
        template = file['title']
    if file['mimeType'] == 'image/png':
        print(file['title'])
        file.GetContentFile(file['title'])
        photo_default = file['title']
    if file['mimeType'] == 'application/vnd.google-apps.spreadsheet':
        file.GetContentFile(file['title'] + '.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        tables.append(file['title'] + '.xlsx')
    if file['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        file.GetContentFile(file['title'])
        tables.append(file['title'])
    if file['mimeType'] == 'text/csv':
        file.GetContentFile(file['title'])
        tables.append(file['title'])
for file_id in videoquest_root_ids:
    file = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % file_id}).GetList()[0]
    file.GetContentFile(file['title'])
    tables.append(file['title'])
tables

Шаблон отчета v.4.0a.docx
photo_default.png


['Game.xlsx',
 'Knowledge.xlsx',
 'Nice2MeetYou.xlsx',
 'QuestionsFromStudents.xlsx',
 'Cases.xlsx',
 'Тестирование. 21 вопрос.xlsx',
 'TestOpinion.xlsx',
 'Вопрос+от+Ивана+Федосеева.xlsx',
 'Вопрос+от+Ольги+Орловой.xlsx',
 'Вопрос+от+Катерины+Ходуновой.xlsx',
 'Вопрос+от+Олега+Лысака.xlsx']

### Таблицы к DataFrame

In [5]:
dfs = []
for table in tables:
    df = pd.read_excel(table, dtype = str, keep_default_na = False)
    if 'Потрачено времени' in df.columns:
        df = df.iloc[1:]
        df['Процент правильных ответов (%)'] = df['Процент правильных ответов (%)'].str.replace(',','.').apply(float).round().apply(int).apply(str)
        df.rename(columns = {'Потрачено времени': 'time', 'Процент правильных ответов (%)': 'score_test'}, inplace = True)
    df.columns = [col.replace(' ', '').replace('/', '').replace('.', '').lower() for col in df.columns]
    dfs.append(df)

### Получение имен, городов

In [6]:
index_table = [tab for tab in tables if tab.startswith('Nice')][0]
df0 = pd.read_excel(index_table, dtype = str, keep_default_na = False)
df0['gorodprojivaniya'].replace('/', ';', regex=True, inplace=True)
df0['ma_email'] = df0['ma_email'].str.lower()
emails = df0['ma_email'][:].unique()
emails

array(['bgdshv.a@gmail.com', 'elashkina.anna@gmail.com',
       'fedoseev.is@enrubattery.ru', ..., 'nitroman779@gmail.com',
       'aurikus1324@gmail.com', 'ilya519697@gmail.com'], dtype=object)

### Колонка почты

In [7]:
def email_col(df):
    return max([col for col in df.columns if 'mail' in col], key=len)

## Первичный балл

### Словарь баллов

In [8]:
primary_keys_dict = {'chtoudivilo': 1, 'chtorazozlilo': 1, 'chtoudivilomaterials': 4, 'chtorazozlilomaterials': 4,
                                'vopros1': 1, 'vopros2': 1, 'vopros3': 1, 'question': 1,
                                'q1videoaudiourls': 2, 'q2videoaudiourls': 2, 'q3videoaudiourls': 2, 'q4videoaudiourls': 2}
primary_cases_list = ['case1', 'case2', 'case3', 'case4']

### Токенизация

In [9]:
# Lemmatizer and stopwords
mystem = Mystem() 
stop_words = set(get_stop_words('russian'))

# Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(re.sub(r'\d+', '', text.lower()))
    tokens = [token for token in tokens if token not in stop_words
              and token != " "
              and token.strip() not in punctuation]
    
    return list(dict.fromkeys(tokens))

def case_counter(text, case):
    tokens = preprocess_text(text)
    
    return sum(word in dict_cases[case] for word in tokens)

### Словари кейсов

In [10]:
dict_cases = {
    'case1': "представлять предприниматель компания заниматься разработка высокоточный станок производство пробный партия необходимо купить специальный клей исследовать возможный предложение интернет понимать поставка организовывать германия необходимый количество стоять евро самостоятельно приходиться сделать следующий заплатить закупка доставка налог пошлина потратить рабочий оформление документ ждать счастие инженер отец жить близкий собираться навещать сын заказывать домой привозить багаж неделя действовать официальный воспользоваться удачный стечение обстоятельство написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case2': "представлять коммерческий директор компания разработка отопительный котел новый поколение месяц выпускать прототип показывать заказчик огромный строительный фирма оставаться довольный сделать следующий предложение готовый купить экземпляр пробный запускать достраивать здание проба продлиться оказываться годный сразу партия знать рынок конкурент каков особенность продукция данный момент участвовать испытание соглашаться отдавать начинать оформлять тз производство штука контрактный главный разработчик сообщать находить решение повышать производительность переделывать опытный образец конструкторский бюро проверять тестовый стенд число поменять короб помещаться отправлять устаревать конструкция инженер говорить срочно предлагать вернуть тестирование утверждать  % ,  вариант причина выбирать стоять вопрос обращаться прерывать середина путь технический характеристика продукт рискнуть потратить деньги подготовка документация улучшать точно скоро хороший примета складываться ситуация написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case3': "представлять молодой предприниматель открывать небольшой стартап партнерство одногруппник разрабатывать крутой приложение смартфон уверять взлетать пользоваться огромный спрос очередной этап разработка понимать свой деньги хватать нужный инвестиция нанимать дополнительный разработчик дизайнер тестировщик покупать лицензия облако место надежный сервер посчитать близкий находить млн рубль неделя поиск приходить кредитный предложение банк предлагать вполне сносный условие взять льготный годовой получать перевод завтра отвечать венчурный фонд mobileventures согласный проинвестировать транш входить доля ждать результат месяц возможный следующий раунд переговоры партнер встречаться двоюродный дядя рассказывать идея понравиться склон скопить приличный сумма заниматься поставка продукт питание из-за рубеж готовый вкладывать единственный поставлять держать курс дело принятие решение интересно следить строиться новый технология поступать источник вообще написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case4': "представлять начальник департамент инновация крупный корпорация подобный ibm усиливать инновационность деятельность собственно продукт достаточно большой бюджет поставлять амбициозный задача ряд проектный сессия вариант деньги выделять хватить начинать инвестировать самостоятельно фонд молодой стартапа нужный направление ожидать выходить улучшать повышать эффективность работа самый инвестиция предполагать получение доля компания должный возвращать последствие либо полностью выкупать процент успешность оценивать предлагать сотрудник сумма экономия приносить идея обязывать гарантировать рассмотрение поступать обоснованный ответ случай отказ принимать  –  таков корпоративный этика решение совещание совет директор".split(' ')
    }

### Словари новеллы

In [11]:
q_scored = ['q23', 'q50', 'q34', 'q41', 'q48', 'q62', 'q70']
q_translated = ['q10', 'q18', 'q33', 'q40', 'q45', 'q48', 'q59', 'q60', 'q61', 'q70']

### ID папки с новеллой

In [12]:
novella_root_id = drive.ListFile({'q': "title = 'НовеллаБД'"}).GetList()[0]['id']

### Загружаем данные о новелле

In [13]:
file_list = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % novella_root_id}).GetList()
for file in file_list:
    if file['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        file.GetContentFile(file['title'])
        rules = pd.read_excel(file['title'])
    if file['mimeType'] == 'text/csv':
        file.GetContentFile(file['title'])
        data = pd.read_csv(file['title'], sep = ";", encoding="windows-1251", quotechar = '"')

### Добавляем данные о новелле

In [14]:
def novella_insert(email, data, rules):
    if data[data['email'] == email]['history_new'].to_list() != []:
        h_str = data[data['email'] == email]['history_new'].to_list()[0]
        h_json = json.loads(h_str)
        h_list = [rec for rec in h_json.values() if rec != {}]
        if h_list == []:
            return(0)
        h_record = max(h_list, key=max)
        novella_score = len(set(h_record.keys()).intersection(q_scored)) * 5
        return(novella_score)
    else:
        return(0)

### Подсчет баллов

In [15]:
def primary_point(context):
    if 'score' in context:
        pts = 0.4 * max([int(num) for num in context['score'].split()])
    else:
        pts = 0
    for key, value in primary_keys_dict.items():
        if key in context.keys() and len(context[key]) > 10:
            pts += value
    for key in primary_cases_list:
        if key in context.keys():
            if len(context[key]) >= 50:
                pts += 1
            if len(context[key]) >= 300:
                pts += 1
            counter = case_counter(context[key], key)
            if counter >= 4:
                pts += 1
            if counter >= 6:
                pts += 1
            if counter >= 8:
                pts += 1
    if context['novel_score'] != '-':
        pts += context['novel_score']
            
    return round(pts)

## Вывод

In [16]:
emails_out = emails[:]
len(emails)

1924

In [17]:
start_time = time.time()

### Общий поиск и запись анкет

In [18]:
keys = re.findall('{{(.+)}}', docx2txt.process(template))
keys.append('q1videoaudiourls')
result = []
progress = 0
url = 'https://10education.ru/CompleteSearch/update_pp.php'

for email in tqdm(emails_out):
    count = 0
    doc = DocxTemplate(template)
    context = {}
    for df in dfs:
        mail = email_col(df)
        df[mail] = df[mail].str.lower().str.replace(' ', '')
        for key in set(df.columns.to_list()).intersection(keys):
            try:
                context[key] = max(df[df[mail] == email][key], key=len)
                if key in ['score_test', 'time']:
                    context[key] = df[df[mail] == email].groupby([mail])[key].apply('  \n  '.join)[0]
                elif key == 'q1videoaudiourls':
                    count += 1
                    context['q' + str(count)] = max(df[df[mail] == email][key], key=len)
            except:
                pass
    try:
        last, name, patronymic = context['name'].split()
    except:
        pass
    context['name'] = max(df0[df0['ma_email'] == email]['name'], key=len)
    context['email'] = email
    context['novel_score'] = novella_insert(email, data, rules)
    context['score'] = primary_point(context)
    context = {'Имя': context['name'], 'Первичный П': context['score']}
    requests.post(url, data = dumps(context))
    
print("Время работы: %s секунд" % round(time.time() - start_time))
print("Длина таблицы: {}".format(len(emails)))
print('Последнее обновление: {}'.format(datetime.datetime.now().strftime("%Y.%m.%d_%H:%M")))

100%|████████████████████████████████████| 1924/1924 [1:02:28<00:00,  1.95s/it]

Время работы: 3749 секунд
Длина таблицы: 1924
Последнее обновление: 2021.10.01_12:57
